# Project 3: Smart Beta Portfolio and Portfolio Optimization

## Overview


Smart beta has a broad meaning, but we can say in practice that when we use the universe of stocks from an index, and then apply some weighting scheme other than market cap weighting, it can be considered a type of smart beta fund.  A Smart Beta portfolio generally gives investors exposure or "beta" to one or more types of market characteristics (or factors) that are believed to predict prices while giving investors a diversified broad exposure to a particular market. Smart Beta portfolios generally target momentum, earnings quality, low volatility, and dividends or some combination. Smart Beta Portfolios are generally rebalanced infrequently and follow relatively simple rules or algorithms that are passively managed.  Model changes to these types of funds are also rare requiring prospectus filings with US Security and Exchange Commission in the case of US focused mutual funds or ETFs.. Smart Beta portfolios are generally long-only, they do not short stocks.

In contrast, a purely alpha-focused quantitative fund may use multiple models or algorithms to create a portfolio. The portfolio manager retains discretion in upgrading or changing the types of models and how often to rebalance the portfolio in attempt to maximize performance in comparison to a stock benchmark.  Managers may have discretion to short stocks in portfolios.

Imagine you're a portfolio manager, and wish to try out some different portfolio weighting methods.

One way to design portfolio is to look at certain accounting measures (fundamentals) that, based on past trends, indicate stocks that produce better results.  


For instance, you may start with a hypothesis that dividend-issuing stocks tend to perform better than stocks that do not. This may not always be true of all companies; for instance, Apple does not issue dividends, but has had good historical performance.  The hypothesis about dividend-paying stocks may go something like this: 

Companies that regularly issue dividends may also be more prudent in allocating their available cash, and may indicate that they are more conscious of prioritizing shareholder interests.  For example, a CEO may decide to reinvest cash into pet projects that produce low returns.  Or, the CEO may do some analysis, identify that reinvesting within the company produces lower returns compared to a diversified portfolio, and so decide that shareholders would be better served if they were given the cash (in the form of dividends).  So according to this hypothesis, dividends may be both a proxy for how the company is doing (in terms of earnings and cash flow), but also a signal that the company acts in the best interest of its shareholders.  Of course, it's important to test whether this works in practice.


You may also have another hypothesis, with which you wish to design a portfolio that can then be made into an ETF.  You may find that investors may wish to invest in passive beta funds, but wish to have less risk exposure (less volatility) in their investments.  The goal of having a low volatility fund that still produces returns similar to an index may be appealing to investors who have a shorter investment time horizon, and so are more risk averse.

So the objective of your proposed portfolio is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  Also, if this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

Smart Beta ETFs can be designed with both of these two general methods (among others): alternative weighting and minimum volatility ETF.


## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.
### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -r requirements.txt

In [2]:
!python -m pip install plotly==2.2.3 --no-cache

In [3]:
# Restart the Kernel
import plotly
print(plotly.__version__)
# Should return plotly==2.2.3

2.2.3


### Load Packages

In [4]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
For this universe of stocks, we'll be selecting large dollar volume stocks. We're using this universe, since it is highly liquid.

In [5]:
df = pd.read_csv('../../data/project_3/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [6]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio
In Part 1 of this project, you'll build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF.  You'll compare this portfolio to a market cap weighted index to see how well it performs. 

Note that in practice, you'll probably get the index weights from a data vendor (such as companies that create indices, like MSCI, FTSE, Standard and Poor's), but for this exercise we will simulate a market cap weighted index.

## Index Weights
The index we'll be using is based on large dollar volume stocks. Implement `generate_dollar_volume_weights` to generate the weights for this index. For each date, generate the weights based on dollar volume traded for that date. For example, assume the following is close prices and volume data:
```
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...
```
The weights created from the function `generate_dollar_volume_weights` should be the following:
```
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...
```

In [7]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function
    d_vol = close * volume
    d_vol_weights = d_vol.div(d_vol.sum(axis=1), axis=0)

    return d_vol_weights

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [8]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

## Portfolio Weights
Now that we have the index weights, let's choose the portfolio weights based on dividend. You would normally calculate the weights based on trailing dividend yield, but we'll simplify this by just calculating the total dividend yield over time.

Implement `calculate_dividend_weights` to return the weights for each stock based on its total dividend yield over time. This is similar to generating the weight for the index, but it's using dividend data instead.
For example, assume the following is `dividends` data:
```
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...
```
The weights created from the function `calculate_dividend_weights` should be the following:
```
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...
```

In [9]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    divs_cum = dividends.cumsum(axis=0)
#     tot_div = divs_cum.sum(axis=1)
    dividend_weights = divs_cum.div(divs_cum.sum(axis=1), axis=0)

    return dividend_weights

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

Tests Passed


### View Data
Just like the index weights, let's generate the ETF weights and view them using a heatmap.

In [10]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

## Returns
Implement `generate_returns` to generate returns data for all the stocks and dates from price data. You might notice we're implementing returns and not log returns. Since we're not dealing with volatility, we don't have to use log returns.

In [11]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
#     print(prices)
    returns = prices.pct_change()

    return returns

project_tests.test_generate_returns(generate_returns)

Tests Passed


### View Data
Let's generate the closing returns using `generate_returns` and view them using a heatmap.

In [12]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF. Implement `generate_weighted_returns` to create weighted returns using the returns and weights.

In [13]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function
    weighted_returns = returns * weights

    return weighted_returns

project_tests.test_generate_weighted_returns(generate_weighted_returns)

Tests Passed


### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [14]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

## Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [15]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function
#     print(returns)
#     returns.dropna(inplace=True)
    cumulative_returns =   (1 + returns.sum(axis = 1)).cumprod()
#     print(cumulative_returns)
    
    return cumulative_returns

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

Tests Passed


### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [16]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [17]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function
    difference = etf_returns_by_date - benchmark_returns_by_date
    tracking_error = np.sqrt(252) * np.std(difference, ddof=1)

    return tracking_error

project_tests.test_tracking_error(tracking_error)

Tests Passed


### View Data
Let's generate the tracking error using `tracking_error`.

In [18]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.10207614832007529


# Part 2: Portfolio Optimization

Now, let's create a second portfolio.  We'll still reuse the market cap weighted index, but this will be independent of the dividend-weighted portfolio that we created in part 1.

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.


## Covariance
Implement `get_covariance_returns` to calculate the covariance of the `returns`. We'll use this to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [19]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function
    ret = returns.fillna(0)
#     print(ret)
    returns_covariance = np.cov(ret.T)
#     print(returns_covariance)
    
    return returns_covariance

project_tests.test_get_covariance_returns(get_covariance_returns)

Tests Passed


### View Data
Let's look at the covariance generated from `get_covariance_returns`.

In [20]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?



### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [21]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    #TODO: Implement function
    m = covariance_returns.shape[0]
    xVar = cvx.Variable(m)
    p_variance = cvx.quad_form(xVar, covariance_returns)
    distance_to_index = cvx.norm(xVar - index_weights)
    objective = cvx.Minimize(p_variance + scale * distance_to_index)
    constraints = [xVar >= 0, sum(xVar) == 1]
    cvx.Problem(objective, constraints).solve()
    x = xVar.value
    print(x)
    
    return x

project_tests.test_get_optimal_weights(get_optimal_weights)

[0.23623897 0.01256285 0.75119817]
Tests Passed


## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [22]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

[0.00505383 0.04996914 0.00507628 0.00344529 0.00631443 0.00723481
 0.00628914 0.00690689 0.05022972 0.00367206 0.00923578 0.00420389
 0.00659224 0.03065369 0.0057022  0.00386121 0.0159118  0.00755899
 0.00230404 0.00784448 0.00719021 0.01067014 0.00471339 0.00600604
 0.01150549 0.00522686 0.01085341 0.00530558 0.00845056 0.00402521
 0.0081877  0.00579648 0.00573072 0.00533293 0.00660151 0.04082251
 0.00271105 0.004656   0.00335664 0.01048722 0.00708355 0.00395319
 0.02907164 0.03255119 0.0105735  0.00460467 0.00836054 0.00353609
 0.00806711 0.01229801 0.01038399 0.02300489 0.00370103 0.00620664
 0.00700668 0.00450263 0.00544879 0.00660128 0.00573769 0.0037471
 0.00505419 0.00563285 0.01461977 0.00793407 0.00560465 0.0250028
 0.03350294 0.01192162 0.04150111 0.040325   0.00993157 0.00412432
 0.00568072 0.00669019 0.00783857 0.00553949 0.00366138 0.00714256
 0.00952813 0.00709645 0.00780563 0.01234318 0.00308681 0.00712188
 0.00539759 0.00398087 0.00669447 0.00567573 0.00357363 0.005318

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [23]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

Optimized ETF Tracking Error: 0.05795012630412267


## Rebalance Portfolio Over Time
The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [24]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    all_rebalance_weights = []

    returns = returns.sort_index()
    index_weights = index_weights.sort_index()
    
    for i in range(chunk_size, len(returns), shift_size):        
        covariance_returns = get_covariance_returns(returns.iloc[i-chunk_size:i])
        new_weights = get_optimal_weights(covariance_returns, index_weights.iloc[i-1])
        all_rebalance_weights.append(new_weights)
        
    return all_rebalance_weights

project_tests.test_rebalance_portfolio(rebalance_portfolio)

[0.29341237 0.41378419 0.29280344]
[0.29654088 0.40731481 0.29614432]
[0.29868214 0.40308791 0.29822995]
[0.30100044 0.39839644 0.30060312]
Tests Passed


Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [25]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

[0.01043134 0.06006002 0.01145332 0.00460319 0.00899074 0.00659741
 0.00588434 0.00543873 0.01913355 0.00440472 0.00117287 0.00498903
 0.00982399 0.02170975 0.00656725 0.00762632 0.01871608 0.00315683
 0.01490108 0.00299942 0.00180181 0.01017089 0.00302964 0.00757446
 0.00280221 0.00379319 0.00894089 0.00609675 0.01479801 0.00855455
 0.01117655 0.00342065 0.00574597 0.0052358  0.0066932  0.06960769
 0.00447415 0.00371878 0.00759784 0.01160758 0.01144284 0.00780362
 0.0217761  0.02548132 0.00738601 0.01097032 0.0083364  0.00317682
 0.0111048  0.01594194 0.01052272 0.01329633 0.00202779 0.01358219
 0.00451156 0.00311991 0.0050552  0.00628738 0.00545589 0.00816001
 0.01323018 0.00557249 0.01257088 0.00812513 0.00680572 0.01944452
 0.01416098 0.0194583  0.00839333 0.00246675 0.0137056  0.00461931
 0.00608005 0.01084456 0.01092437 0.01692651 0.0077317  0.0113174
 0.00577372 0.00556129 0.03248384 0.00983902 0.00390546 0.00569
 0.00331695 0.00602821 0.00490272 0.00619657 0.00309221 0.00749353

[0.00560596 0.10550724 0.0113689  0.00240813 0.00629966 0.00939865
 0.00400875 0.00617093 0.01851353 0.00604411 0.00284837 0.00589542
 0.00695929 0.04141948 0.00535906 0.00541038 0.02481997 0.01101844
 0.00470616 0.00608747 0.00123899 0.01039187 0.00613465 0.00657478
 0.00382823 0.03824855 0.01226563 0.0065066  0.00890404 0.00647026
 0.01028806 0.00647152 0.00541522 0.00749407 0.00712437 0.03930448
 0.00363578 0.00393783 0.005492   0.01317931 0.03216955 0.00635116
 0.01157497 0.0131243  0.01126325 0.00818105 0.01464855 0.0032168
 0.00814706 0.01676907 0.01192334 0.0136607  0.00859617 0.01025624
 0.00448204 0.00317228 0.00629816 0.00470017 0.00759077 0.0034889
 0.00760502 0.00348443 0.00947376 0.00683023 0.00706921 0.0190399
 0.00988375 0.02374164 0.00554451 0.00292125 0.00920171 0.00642978
 0.00796453 0.01037311 0.01302766 0.00672461 0.00330677 0.00861346
 0.00386016 0.00416601 0.01035156 0.01023619 0.00620679 0.00566696
 0.00352159 0.00413666 0.00414964 0.005631   0.00567616 0.0038150

[0.0073259  0.11513997 0.01372478 0.00306075 0.00600021 0.0067101
 0.00309138 0.01322496 0.01579436 0.00887989 0.00167446 0.00660426
 0.00634951 0.01875692 0.00691835 0.00430193 0.01071043 0.00569518
 0.00712997 0.01169055 0.00332609 0.01085923 0.01245976 0.00664995
 0.00639474 0.00339511 0.01110184 0.00691603 0.01343251 0.0107753
 0.01070688 0.00385179 0.00636091 0.00249988 0.00347221 0.04251673
 0.00448859 0.00407962 0.00732695 0.01192331 0.01789252 0.00483483
 0.02160739 0.02330954 0.0062133  0.0152439  0.00758    0.00594866
 0.05504438 0.01376301 0.01138918 0.00887241 0.0045421  0.01076323
 0.00263752 0.0051807  0.00471453 0.00366919 0.00705714 0.00339014
 0.01171896 0.00337051 0.00664181 0.00921666 0.00545994 0.02212493
 0.01056929 0.02740382 0.00456575 0.00135271 0.00950408 0.00438227
 0.00920525 0.01510396 0.01016995 0.00648915 0.00389228 0.00665425
 0.00668749 0.00445174 0.0112112  0.00694017 0.00364173 0.00584852
 0.00610436 0.00650533 0.00650802 0.00592698 0.0042128  0.003608

[0.01328943 0.07912487 0.00638761 0.0032239  0.01082637 0.00504433
 0.00750054 0.00851275 0.02490557 0.00963706 0.00262216 0.0053425
 0.01160406 0.01980751 0.00714505 0.00518344 0.0222652  0.00830827
 0.0046233  0.01005671 0.00163831 0.01051534 0.00341734 0.00955859
 0.00393801 0.00413961 0.01117788 0.01134251 0.01452671 0.01092401
 0.00734853 0.00434023 0.0089849  0.00865186 0.00657239 0.03059949
 0.00617665 0.00612952 0.00591639 0.01358816 0.01573279 0.00836625
 0.02542752 0.02813426 0.00949201 0.01324177 0.00915418 0.00467288
 0.01172313 0.01298912 0.01237278 0.01430349 0.01202119 0.01096789
 0.00403837 0.00268823 0.00559825 0.0072899  0.01571014 0.00502977
 0.00717877 0.00479225 0.00517273 0.0073947  0.00493623 0.0257492
 0.01222713 0.00996667 0.00848323 0.00112814 0.0082151  0.00736386
 0.00755788 0.00829717 0.00909651 0.0075189  0.00664045 0.00927608
 0.00477803 0.00678476 0.01519944 0.01006693 0.00656322 0.00653425
 0.00446975 0.00512997 0.00570494 0.00648677 0.00627195 0.004668

[0.00762813 0.10682203 0.00958469 0.00459254 0.01025325 0.01646455
 0.00490423 0.00833253 0.02179727 0.00796979 0.00248977 0.04227273
 0.00768445 0.02428266 0.00699858 0.00466404 0.0141659  0.00708445
 0.01573474 0.01447645 0.00168591 0.01333722 0.00418451 0.00878426
 0.00431392 0.00527275 0.02180547 0.00650302 0.01212556 0.01128027
 0.00848945 0.00375369 0.00646267 0.00292598 0.00442101 0.02316952
 0.00638772 0.00471907 0.00664801 0.01218234 0.01715445 0.00784013
 0.01709177 0.02145026 0.00805616 0.01206284 0.00869465 0.00312393
 0.00864878 0.00925641 0.01657569 0.01480087 0.00524479 0.00663251
 0.00286528 0.00368798 0.00286435 0.00770489 0.00959371 0.00431616
 0.01034471 0.00468053 0.00530318 0.00682596 0.00454016 0.02665894
 0.00980457 0.0141856  0.00558164 0.00275461 0.00696837 0.00529856
 0.00829891 0.01181606 0.00796301 0.00574425 0.00884983 0.00828285
 0.00531154 0.00435695 0.01005988 0.00773878 0.00266391 0.00445218
 0.0046734  0.00474786 0.00634254 0.00626162 0.00317058 0.0058

...

[0.00432235 0.15711999 0.0071763  0.00372779 0.01379691 0.00573196
 0.00712704 0.00744207 0.03618439 0.00605374 0.00932167 0.00445207
 0.00655579 0.0159929  0.00550371 0.00806537 0.01005807 0.0042484
 0.00223245 0.00578272 0.00696254 0.0100124  0.00723432 0.00465812
 0.00511899 0.00490919 0.01177504 0.00997468 0.00970173 0.00509493
 0.01056427 0.00370412 0.00547896 0.00367387 0.00965566 0.02995991
 0.00612511 0.0034161  0.0062931  0.01529406 0.01174143 0.00603813
 0.0160545  0.01938546 0.00418095 0.00430932 0.00963551 0.00563601
 0.00822615 0.01323762 0.01017502 0.01243756 0.00442749 0.00631751
 0.00308414 0.00450023 0.00420127 0.00529788 0.00629756 0.00446812
 0.00595131 0.00529278 0.01654935 0.00823742 0.00479467 0.02302628
 0.00655565 0.00796872 0.00529079 0.01239517 0.01170609 0.00550447
 0.00603456 0.01028121 0.01787102 0.00555694 0.00436843 0.00804083
 0.00503781 0.00652941 0.00588283 0.01311793 0.00537612 0.00675478
 0.00642209 0.00377519 0.00680358 0.00469977 0.00337375 0.00667

[0.00275929 0.04585039 0.00737644 0.00291323 0.01235691 0.00583269
 0.00312313 0.00982987 0.07215794 0.00161161 0.00630802 0.00490304
 0.00705566 0.04245143 0.00607817 0.00731532 0.02079088 0.00971049
 0.00183648 0.01110186 0.01076287 0.01318071 0.00653361 0.00369845
 0.00416808 0.00572684 0.00855101 0.00717442 0.006713   0.00704319
 0.00969968 0.00261016 0.00208603 0.00511896 0.00437657 0.06249507
 0.00864705 0.00316663 0.00267086 0.01779776 0.00939907 0.00647292
 0.02759161 0.03516129 0.01699583 0.00417293 0.00826463 0.0054537
 0.00888495 0.00714021 0.01036202 0.03162138 0.00360932 0.00977958
 0.00396297 0.00493142 0.00502511 0.00519233 0.00370163 0.00619469
 0.00388093 0.00670134 0.00244541 0.00837942 0.01087119 0.02492819
 0.00392311 0.01735439 0.00462647 0.01115674 0.00787025 0.00426009
 0.00680011 0.01622346 0.01332274 0.0095755  0.00205638 0.00841832
 0.00461376 0.00537945 0.00394205 0.00981128 0.00485103 0.00505398
 0.00365438 0.00261256 0.00659796 0.0064033  0.00373725 0.00610

[0.00592083 0.04702999 0.00654859 0.00611767 0.00998014 0.00532654
 0.00472403 0.00836346 0.03366198 0.00470801 0.00871503 0.01184719
 0.00756595 0.03740615 0.00639098 0.00581325 0.01311508 0.00593698
 0.00612116 0.00929876 0.00744424 0.01399923 0.02202342 0.00433228
 0.00373118 0.00488572 0.01005786 0.00926725 0.01109358 0.00688124
 0.01393948 0.00691774 0.00493124 0.00524651 0.0113644  0.03561449
 0.01134127 0.00417307 0.00408164 0.0134153  0.01052543 0.02025946
 0.01565654 0.01635151 0.01350875 0.00862156 0.01289247 0.00709374
 0.01062269 0.0092388  0.00981006 0.019352   0.00500957 0.01270844
 0.00431208 0.00467661 0.00473103 0.00556382 0.00594911 0.00713681
 0.00919675 0.00551685 0.00285044 0.01446495 0.00627738 0.01866185
 0.00767212 0.01285336 0.00692745 0.03856708 0.0070895  0.00427002
 0.00870836 0.01012644 0.01436425 0.00590868 0.00415018 0.00622368
 0.01102209 0.00620045 0.00576716 0.0136052  0.00521578 0.00797844
 0.0044136  0.00431704 0.00914983 0.00589639 0.00519867 0.0042

[0.00699178 0.05267159 0.00751278 0.00442396 0.00827435 0.00916248
 0.00548484 0.00756296 0.03054292 0.0061556  0.00963868 0.00619984
 0.00950486 0.0342921  0.006564   0.00733741 0.02258974 0.01434549
 0.00288493 0.01224911 0.0052054  0.00712872 0.00389826 0.00572532
 0.00547284 0.00530949 0.00991261 0.00736134 0.0132857  0.00705892
 0.00765517 0.00321685 0.0082809  0.00774182 0.00589836 0.03754431
 0.00462659 0.00464902 0.00351425 0.01289299 0.01555666 0.00778948
 0.01849615 0.02021112 0.0131737  0.01304721 0.00896011 0.00533554
 0.01602046 0.01430997 0.01826515 0.02127416 0.00433839 0.01012428
 0.00838064 0.00332292 0.00527667 0.00613762 0.00709597 0.00552328
 0.00486833 0.00671197 0.00440492 0.00850262 0.00887138 0.02190676
 0.00957896 0.00651712 0.00597883 0.02051021 0.00749689 0.00844085
 0.00746705 0.01088198 0.00847758 0.0062767  0.01030297 0.00953683
 0.00751387 0.01679697 0.0124756  0.01214117 0.0099953  0.00494353
 0.00571114 0.00473291 0.01047656 0.00746408 0.0040968  0.0048

[0.00436082 0.07236544 0.00538906 0.00467589 0.00568977 0.00670026
 0.00355802 0.00851352 0.04854336 0.00585802 0.00922591 0.00409414
 0.00858914 0.02389791 0.00909682 0.00574469 0.01232018 0.00721568
 0.00549807 0.008678   0.01207155 0.01218799 0.00830362 0.00651797
 0.01149163 0.00409273 0.00903092 0.00529201 0.00924771 0.00452765
 0.01173662 0.00519154 0.00768614 0.00392658 0.00548231 0.04670783
 0.0038374  0.00384995 0.00806081 0.00859519 0.00686127 0.00591546
 0.03212482 0.02783225 0.00985677 0.00964057 0.0068654  0.01030293
 0.03343826 0.01178935 0.00897114 0.01449177 0.00350314 0.00617688
 0.00453662 0.00390127 0.00463057 0.00494418 0.00620848 0.00576693
 0.00596552 0.00549819 0.0035308  0.01041952 0.00528007 0.0233401
 0.00799562 0.0151653  0.01287942 0.01071178 0.0068685  0.00797111
 0.00602312 0.01263416 0.00684108 0.00556196 0.00744667 0.00864358
 0.00824876 0.00694608 0.01397263 0.01447106 0.00479436 0.00649195
 0.00514913 0.00295713 0.00678616 0.00601531 0.00416091 0.00490

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover. We'll be using the formulas used in the classroom:

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [26]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    total_turnover = .0
    
    
    for i in range(1, len(all_rebalance_weights)):
        t_over = np.sum(np.abs(all_rebalance_weights[i] - all_rebalance_weights[i - 1]))
        total_turnover += t_over
    
    avg_turnover = total_turnover / rebalance_count
    portfolio_turnover = (avg_turnover * (n_trading_days_in_year / shift_size))
    
    return portfolio_turnover

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)

Tests Passed


Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [27]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

16.72683266050277


That's it! You've built a smart beta portfolio in part 1 and did portfolio optimization in part 2. You can now submit your project.

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.